In [3]:
# pip install tabula-py

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# pip install tabulate

In [1]:
from tabula import read_pdf
from tabulate import tabulate
from os import listdir
from os.path import isfile, join
import pandas as pd

In [129]:
def get_raw_prediction_dfs(path):
    dfs = read_pdf(
        path,
        pages='all', 
        lattice=True,
        pandas_options = {'header':None},
        multiple_tables = True,
        silent=True
    )
    
    # usually data df has many columns, tunable. 
    valid_col_num = 7
    valid_dfs = filter(lambda df: df.shape[-1] > valid_col_num, dfs)
    raw_prediction_dfs = list(map(lambda df: df.dropna(how='all'), valid_dfs))
    return raw_prediction_dfs

In [4]:
raw_prediction_dfs = get_raw_prediction_dfs('./data_es/LikeFolioSundayEarningsSheet20210313.pdf')
raw_prediction_dfs[0]

Got stderr: 九月 07, 2021 1:52:21 下午 org.apache.fontbox.ttf.CmapSubtable processSubtype14



,0,1,2,3,4,5,6,7,8,9,10,11,12
0,NaN,LikeFolio Sunday Earnings Sheet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Week 9\r2021-03-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Company,Earnings\rScore,Buzz\rGrowth,Demand\rGrowth,Happiness\rGrowth,YoY Change,Expected\rMove,Short\rInterest,Andy's\rTrade,Landon's\rTrade,System\rTrade,Commentary
3,NaN,Designer Brands\rDBI\r$15.36\rTue before market,-51,NaN,NaN,NaN,PI: -48.2%\rStock: +65.4%,+ or -\r$2.20\r(14.3%),14.5%\rShort\r(02/26/21),Buy OTM\rPuts,Very\rBearish\rSpread,Coin Flip\rBearish,The re-opening narrative has been pushing the ...
5,NaN,Kodak\rKODK\r$9.76\rTue after market,+65,NaN,NaN,NaN,PI: +3.1%\rStock: +378.4%,+ or -\r$2.47\r(25.3%),22.9%\rShort\r(02/26/21),NaN,NaN,Very\rBullish\rSpread,"Got squeezed to high heaven last July, followi..."
6,NaN,Lands End\rLE\r$35.49\rWed before market,+17,NaN,NaN,NaN,PI: +15.0%\rStock: +538.3%,+ or -\r$4.61\r(13.0%),11.6%\rShort\r(02/26/21),NaN,NaN,NaN,Weak bullish Score with a mixed signal and low...
8,NaN,Dollar General\rDG\r$192.74\rThu before market,-1,NaN,NaN,NaN,PI: -18.0%\rStock: +36.8%,+ or -\r$8.91\r(4.6%),1.5%\rShort\r(02/26/21),Sell Calls\rAbove,Sell Calls\rAbove,NaN,"DG's expansion across rural America, combined ..."
9,Five Below\rFIVE\r$190.80\rWed after market,-32,NaN,NaN,NaN,PI: +2.5%\rStock: +164.9%,+ or -\r$13.45\r(7.0%),5.9%\rShort\r(02/26/21),Coin Flip\rBearish,High-\rProb.\rBearish\rSpread,High-\rProb.\rBearish\rSpread,"FIVE has been on an impressive run, but market...",NaN


In [5]:
LINE_SEPARATOR = '\r'

In [72]:
import datetime as dt
def get_date(data):
    fmts = ('%Y','%b %d, %Y','%b %d, %Y','%B %d, %Y','%B %d %Y','%m/%d/%Y','%m/%d/%y','%b %Y','%B%Y','%b %d,%Y', 
            '%m-%d-%Y', '%Y-%m-%d')
    for fmt in fmts:
        try:
            t = dt.datetime.strptime(data, fmt)
            date = str(t).split(' ')[0]
            return date
        except:
            pass
    return None

In [73]:
get_date('2021-03-15')

'2021-03-15'

In [89]:
from enum import Enum
class ColType(Enum):
    MORE_LINE_TICKER = 1
    THREE_LINE_TICKER = 2
    NAN = 3
    DATE = 4
    OTHER = 5
    
def col_type(col):
    if pd.isna(col):
        return ColType.NAN
    else:
        values = col.split(LINE_SEPARATOR)        
        if len(values) == 3 and values[1].isupper() and values[2].startswith('$'):
            return ColType.THREE_LINE_TICKER
        elif len(values) in [4, 5] and values[1].isupper() and values[2].startswith('$'):
            return ColType.MORE_LINE_TICKER
        elif len(values) == 2 and get_date(values[1]) :
            return ColType.DATE
        else:
            return ColType.OTHER

In [75]:
test_cols = [float('nan'), 'Kodak\rKODK\r$9.76\rTue after market', 'Week 9\r2021-03-15']
for col in test_cols:
    print(col_type(col))

ColType.NAN
ColType.MORE_LINE_TICKER
ColType.DATE


In [126]:
def get_week_and_df_builder(raw_df):
    stop_token = 'Unconfirmed Earnings'
    is_unconfirmed = False
    df_builder = []
    week = ''
    for idx, row in raw_df.iterrows():
#         print(row.tolist())
        if is_unconfirmed:
            break
        row_builder = []
        cols = row.tolist()
        # pointer to col
        idx = 0
        while idx < len(cols):
            cur_col = cols[idx]
            cur_type = col_type(cur_col)
#             print(cur_type)
            if cur_type == ColType.DATE:
                week = get_date(cur_col.split(LINE_SEPARATOR)[-1])
                break
            elif cur_type == ColType.THREE_LINE_TICKER:
                values = cur_col.split(LINE_SEPARATOR)
                # Company, ticker
                row_builder = [values[0], values[1]]
                # Get weekday info 
                row_builder.append(cols[idx + 1])
                # Get earning score
                row_builder.append(cols[idx + 2])
                df_builder.append(row_builder)
                break
            elif cur_type == ColType.MORE_LINE_TICKER:
                values = cur_col.split(LINE_SEPARATOR)
                # Company, ticker, weekday
                row_builder = [values[0], values[1], values[3]]
                # Get earning score
                row_builder.append(cols[idx + 1])
                df_builder.append(row_builder)
                break
            elif stop_token in str(cur_col):
                is_unconfirmed = True
                break
            idx += 1
        
    
    return week, df_builder, is_unconfirmed

In [127]:
def earning_prediction_df_from_file(path):
    raw_prediction_dfs = get_raw_prediction_dfs(path)
    week_value = None 
    final_df_builder = []
    for df in raw_prediction_dfs:
        week, df_builder, is_following_unconfirmed = get_week_and_df_builder(df)
        if week:
            week_value = week
        final_df_builder.extend(df_builder)
        if is_following_unconfirmed:
            break
    final_df = pd.DataFrame(final_df_builder, columns = ['Company', 'Ticker', 'Earning Time', 'Earning Score'])
    final_df['Starting Week'] = week_value
    
    return final_df

In [133]:
earning_prediction_df_from_file('./data_es\/LikeFolioSundayEarningsSheet_20210509.pdf')

,Company,Ticker,Earning Time,Earning Score,Starting Week
0,Nautilus,NLS,Mon after market,+38,2021-05-10
1,Callaway,ELY,Mon after market,+24,2021-05-10
2,Affirm,AFRM,Mon after market,+12,2021-05-10
3,Hanes,HBI,Tue before market,-11,2021-05-10
4,The RealReal,REAL,Mon after market,-18,2021-05-10
5,Wix,WIX,Wed before market,+25,2021-05-10
6,fuboTV,FUBO,Tue after market,+18,2021-05-10
7,Wendy's,WEN,Wed before market,-10,2021-05-10
8,Toyota,TM,Wed before market,-14,2021-05-10
9,Opendoor,OPEN,Tue after market,-23,2021-05-10


In [130]:
DATA_FOLDER = './data_es'
PATHS = [join(DATA_FOLDER, f) for f in listdir(DATA_FOLDER) if isfile(join(DATA_FOLDER, f))]
earning_prediction_dfs = []
merged_df = None
success_counter = 0
for path in PATHS:
    print(f'Processing {path} ...')
    try:
        earning_prediction_df = earning_prediction_df_from_file(path)
        print(earning_prediction_df.shape)
        if earning_prediction_df.shape[0] > 0: 
            earning_prediction_dfs.append(earning_prediction_df)
            success_counter += 1
        else:
            print(f'[Warning]: Empty dataframe extracted {path}')
    except:
        print(f'Cannot process {path}')
if len(earning_prediction_dfs) > 0:
    merged_df = pd.concat(earning_prediction_dfs)
print(f'Processed {success_counter} files out of {len(PATHS)} raw files')

merged_df

Processing ./data_es\LikeFolioSundayEarningsSheet-02-21-21.pdf ...
(34, 5)
Processing ./data_es\LikeFolioSundayEarningsSheet20200120.pdf ...
(11, 5)
Processing ./data_es\LikeFolioSundayEarningsSheet20210313.pdf ...
(8, 5)
Processing ./data_es\LikeFolioSundayEarningsSheet_20200112_2.pdf ...
(3, 5)
Processing ./data_es\LikeFolioSundayEarningsSheet_20200124.pdf ...
(28, 5)
Processing ./data_es\LikefolioSundayEarningsSheet_20200202.pdf ...
(35, 5)
Processing ./data_es\LikeFolioSundayEarningsSheet_20200209.pdf ...
(23, 5)
Processing ./data_es\LikeFolioSundayEarningsSheet_20200216.pdf ...
(16, 5)
Processing ./data_es\LikeFolioSundayEarningsSheet_20200223.pdf ...
(34, 5)
Processing ./data_es\LikeFolioSundayEarningsSheet_20200301.pdf ...
(17, 5)
Processing ./data_es\LikeFolioSundayEarningsSheet_20200308.pdf ...
(12, 5)
Processing ./data_es\LikeFolioSundayEarningsSheet_20200315.pdf ...
(12, 5)
Processing ./data_es\LikeFolioSundayEarningsSheet_20200412.pdf ...
(5, 5)
Processing ./data_es\LikeFol

,Company,Ticker,Earning Time,Earning Score,Starting Week
0,Macy's,M,Tue before market,+29,2021-02-22
1,IHG Hotels,IHG,Tue before market,+20,2021-02-22
2,Home Depot,HD,Tue before market,-7,2021-02-22
3,Crocs,CROX,Tue before market,-34,2021-02-22
4,Arlo Technologies,ARLO,Tue after market,+36,2021-02-22
...,...,...,...,...,...
7,Lands End,LE,Thu before market,-54,2021-08-30
8,Chewy,CHWY,Wed after market,-59,2021-08-30
9,Tilly's,TLYS,Thu after market,+32,2021-08-30
10,Oxford,OXM,Thu after market,-2,2021-08-30


In [149]:
# Further filter to remove data of unconfirmed date
print(f'Before trimming there are {merged_df.shape[0]} rows')
df_trim_unconfirmed = merged_df[merged_df['Earning Time'].apply(len) > 5] 
final_df = df_trim_unconfirmed.drop_duplicates()
print(f'After trimming there are {final_df.shape[0]} rows')


Before trimming there are 1305 rows
After trimming there are 1299 rows


In [150]:
final_df.to_csv('earning_prediction.csv', index=False)